In [ ]:
# Training Code
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define dataset directories (replace with your actual paths)
training_path = 'path/to/plantvillage/train'
validation_path = 'path/to/plantvillage/validation'

# Enhanced data augmentation for better generalization
train_augmentor = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_augmentor = ImageDataGenerator(rescale=1./255)

train_data = train_augmentor.flow_from_directory(training_path, target_size=(224, 224), batch_size=64, class_mode='categorical')
val_data = val_augmentor.flow_from_directory(validation_path, target_size=(224, 224), batch_size=64, class_mode='categorical')

# Modified CNN architecture with an extra layer
disease_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),  # Added layer for deeper feature extraction
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),  # Adjusted dropout
    Dense(38, activation='softmax')  # Output for 38 classes
])

disease_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to avoid overfitting
stop_early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train with modified epochs and callbacks
training_history = disease_model.fit(train_data, epochs=25, validation_data=val_data, callbacks=[stop_early])

# Save the trained model
disease_model.save('custom_plant_model.h5')


In [ ]:
# Testing code
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the custom model
loaded_model = tf.keras.models.load_model('custom_plant_model.h5')

# Sample class labels (expand to all 38 from your dataset)
disease_classes = ['Apple_healthy', 'Tomato_early_blight', 'Potato_late_blight']  # Add all classes here

# Function for prediction with top-3 results
def predict_disease(img_path, use_grayscale=False):
    img = image.load_img(img_path, target_size=(224, 224), color_mode='grayscale' if use_grayscale else 'rgb')
    img_tensor = image.img_to_array(img) / 255.0
    img_tensor = np.expand_dims(img_tensor, axis=0)

    preds = loaded_model.predict(img_tensor)[0]
    top_indices = np.argsort(preds)[-3:][::-1]  # Get top 3 predictions
    results = [(disease_classes[i], preds[i] * 100) for i in top_indices]

    return results

# Example usage
test_img = 'path/to/sample_leaf.jpg'
predictions = predict_disease(test_img)
for label, confidence in predictions:
    print(f"{label}: {confidence:.2f}% confidence")
